In [1]:
from pathlib import Path
from osgeo import gdal
import rasterio
from rio_tools import get_geopandas_features_from_array
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
from rasterio.crs import CRS

# Tile Paths

In [2]:
tile_dir = Path('annual_avg_coherence_tiles')
tile_dir.exists()

True

In [3]:
tile_paths = list(tile_dir.glob('*.tif'))
tile_paths_str = list(map(str, tile_paths))
tile_paths_str[:2]

['annual_avg_coherence_tiles/N58W134.tif',
 'annual_avg_coherence_tiles/N45W117.tif']

# VRT

In [4]:
vrt_path = str(tile_dir / '__north_america_mask.vrt')

In [5]:
ds = gdal.BuildVRT(vrt_path, tile_paths_str)
ds = None

/Users/cmarshak/mambaforge/envs/s1_coherence/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


# Generate Vector Data

In [6]:
def generate_vector_tile_for_tile(path: Path) -> gpd.GeoDataFrame:
    with rasterio.open(path) as ds:
        X = ds.read(1)
        p = ds.profile
    if X.sum() == 0:
        return gpd.GeoDataFrame()
    features = get_geopandas_features_from_array(X, 
                                                 p['transform'], 
                                                 mask=(~X.astype(bool)))
    df_tile = gpd.GeoDataFrame.from_features(features)[['geometry']]
    df_tile['tile_id'] = path.stem
    return df_tile

In [7]:
df_tile = generate_vector_tile_for_tile(tile_paths[100])

In [8]:
df_tiles = list(map(generate_vector_tile_for_tile, tqdm(tile_paths)))

100%|███████████| 1907/1907 [01:55<00:00, 16.45it/s]


In [9]:
df_tiles_f = list(filter(lambda df_t: not df_t.empty, df_tiles))
len(df_tiles_f)

1733

In [10]:
df_mask = pd.concat(df_tiles_f, axis=0)
df_mask = df_mask.set_crs(CRS.from_epsg(4326))
df_mask.head()

,geometry,tile_id
0,"POLYGON ((-133.16250 57.91333, -133.16250 57.9...",N58W134
1,"POLYGON ((-133.32333 57.87917, -133.32333 57.8...",N58W134
2,"POLYGON ((-133.61583 57.57750, -133.61583 57.5...",N58W134
3,"POLYGON ((-133.13667 57.57250, -133.13667 57.5...",N58W134
4,"POLYGON ((-133.07250 57.53167, -133.07250 57.5...",N58W134


In [11]:
total_area = df_mask.to_crs(CRS.from_epsg(3395)).area.sum()
total_area_km2 = total_area / 10**6

In [12]:
f'{total_area_km2:,}'

'1,145,307.382295196'

There are too many fine geometries. Could just buffer them/

In [14]:
# df_mask.to_file('coherence_mask.geojson')